## US Hospital Readmission and Mortality Rates


**Made by:** Joshua Yu (uqj7ur@virginia.edu) and Esther Olulana (ewo9rkn@virginia.edu)
<br>
*Node Spring 2023*






**About this project:** This project looks into data regarding readmission and mortality rates across the US. Across all states and territories (54), healthcare providers have varying levels of service for different conditions. This project looks into trends between readmission and mortality for conditions, and also external factors impacting healthcare providers.

In [1]:
import pandas as pd
import numpy as np

url= "https://raw.githubusercontent.com/jjyu130/node23-readmission-mortality/main/Readmissions_and_Deaths_-_Hospital.csv"
hospital= pd.read_csv(url)

url2= "https://raw.githubusercontent.com/jjyu130/node23-readmission-mortality/main/MedianIncome.csv"
income = pd.read_csv(url2)

url3= "https://raw.githubusercontent.com/jjyu130/node23-readmission-mortality/main/PopulationEstimates.csv"
pop = pd.read_csv(url3)

url4= "https://raw.githubusercontent.com/jjyu130/node23-readmission-mortality/main/PovertyEstimates.csv"
pov = pd.read_csv(url4)

url5= "https://raw.githubusercontent.com/jjyu130/node23-readmission-mortality/main/Payment_and_Value_of_Care-Hospital.csv"
pay = pd.read_csv(url5)

url6 = "https://raw.githubusercontent.com/jjyu130/node23-readmission-mortality/main/PopulationDensity.csv"
popDen = pd.read_csv(url6)

In [2]:
pd.options.display.max_rows = 5

For this project we used data from the following datasets: <br>
https://www.kaggle.com/datasets/thedevastator/us-healthcare-readmissions-and-mortality?select=Readmissions_and_Deaths_-_Hospital.csv

https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/

https://data.cms.gov/provider-data/dataset/c7us-v4mf

https://covid19.census.gov/datasets/21843f238cbb46b08615fc53e19e0daf_1/explore?showTable=true


The following conditions are included in the dataset:
* Pneumonia (PN)
* Stroke (STK)
* Heart failure (HF)
* Acute myocardial infarction (AMI)
* Coronary artery bypass graft (CABG)
* Chronic obstructive pulmonary disease (COPD)

It also includes **readmission** information regarding specifically hospital-wide readmission and readmission after hip/knee surgery.

## First, Data Preparation!

### A Glimpse Into Our Data Set

In [3]:
hospital.head(2)

,index,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure Name,Measure ID,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Measure Start Date,Measure End Date,Location
0,0,230100,TAWAS ST JOSEPH HOSPITAL,200 HEMLOCK,TAWAS CITY,MI,48764,IOSCO,9893629301,Rate of readmission after discharge from hospi...,READM_30_HOSP_WIDE,No Different than the National Rate,438,13.9,12.6,15.6,NaN,07/01/2014,06/30/2015,"200 HEMLOCK\nTAWAS CITY, MI 48764\n(44.274911,..."
1,1,230121,MEMORIAL HEALTHCARE,826 WEST KING STREET,OWOSSO,MI,48867,SHIAWASSEE,9897235211,Rate of readmission after hip/knee replacement,READM_30_HIP_KNEE,No Different than the National Rate,150,4.0,2.8,5.7,NaN,07/01/2012,06/30/2015,"826 WEST KING STREET\nOWOSSO, MI 48867\n(43.00..."


As seen above, the dataset displays:
1. healthcare provider information ("Provider ID", Location Data, Contact)<br>
2. condition and measure type ("Measure Name" and "Measure ID")
3. measure of quality ("Compared to National")
4. an average rating of service ("Score") <br>

*note: Provider ID is the CMS Certification Number.*

We don't need the columns index and footnote, so we'll drop them and add extra columns based on the rate type and condition.

In [4]:
hospital = hospital.drop(labels = {"index","Footnote","Phone Number"}, axis = 1)

In [5]:
hospital['rateType'] = hospital['Measure ID'].str.split(pat = "_", expand = True)[0]
hospital['condition'] = hospital['Measure ID'].str.split(pat = "_", expand = True)[2]

Also while some other columns help, they are not being used in the project. So we will drop the following columns:

*   Lower Estimate
*   Higher Estimate
*   Measure Start Date
*   Measure End Date



In [6]:
hospital = hospital.drop(columns = {'Lower Estimate', 'Higher Estimate', 'Measure Start Date', 'Measure End Date'}, axis = 1)

We'll also rename some columns for the sake of future use.

In [7]:
hospital.columns = ["providerID", "hospital_name", "address","city","state","zip_code","county_name", "measureName","measureID","compared_to_NR",
                    "denominator","score","location","rateType","condition"]

After all these changes, our data frame now looks like this!

In [8]:
hospital.head(2)

,providerID,hospital_name,address,city,state,zip_code,county_name,measureName,measureID,compared_to_NR,denominator,score,location,rateType,condition
0,230100,TAWAS ST JOSEPH HOSPITAL,200 HEMLOCK,TAWAS CITY,MI,48764,IOSCO,Rate of readmission after discharge from hospi...,READM_30_HOSP_WIDE,No Different than the National Rate,438,13.9,"200 HEMLOCK\nTAWAS CITY, MI 48764\n(44.274911,...",READM,HOSP
1,230121,MEMORIAL HEALTHCARE,826 WEST KING STREET,OWOSSO,MI,48867,SHIAWASSEE,Rate of readmission after hip/knee replacement,READM_30_HIP_KNEE,No Different than the National Rate,150,4.0,"826 WEST KING STREET\nOWOSSO, MI 48867\n(43.00...",READM,HIP


We'll also check the overall data itself.

In [9]:
print(hospital.shape[0])

64764


In [10]:
print(hospital['providerID'].nunique())

4626


> As can be seen, we can determine that the data has **64764** rows and was collected from **4626** different health care providers.



### Removing Data with "Not Available" Rates

When looking at our data set, one issue we came across was that some data were not available. Now we'll remove them.

In [11]:
hospital = hospital[hospital.score != "Not Available"]
print(hospital.shape[0])

41785


In [12]:
print(hospital['providerID'].nunique())

4423


> Removing the "Not Available" rows decreases the number of health care providers used to **4423**, and the number of rows to **41785**.

### Additional Adjustment to Main Data Set

First, for some reason, the 'score' and 'Denominator' columns were in type object, so we converted them to numeric for future use.

In [13]:
pd.set_option('mode.chained_assignment',None)
hospital.score = pd.to_numeric(hospital.score)
hospital.denominator = pd.to_numeric(hospital.denominator)

Next, the "compared_to_NR" column values are too long so we'll simplify them. <br> If the rate is no different than the rate, the value is 0. If better, then 1, and if worse, then -1.

In [14]:
hospital.loc[hospital["compared_to_NR"] == "No Different than the National Rate", "compared_to_NR"] = 'No Different'
hospital.loc[hospital["compared_to_NR"] == "Worse than the National Rate", "compared_to_NR"] = 'Worse'
hospital.loc[hospital["compared_to_NR"] == "Better than the National Rate", "compared_to_NR"] = 'Better'

Also, while the 30 in the measureID is important to indicate 30 days, it can be cumbersome to type and read. <br>
Let's rewrite the measureIDs without the 30.

In [15]:
hospital["measureID"] = hospital["measureID"].str.replace('_30', '')

Let's also transform the location column into latitude and longitude!

In [16]:
hospital['location'] = hospital['location'].str.split(pat = "\n", expand = True)[2]

In [17]:
hospital['lat'] = hospital['location'].str.split(pat = ", ", expand = True)[0].str.replace('(', '')
hospital['long'] =  hospital['location'].str.split(pat = ", ", expand = True)[1].str.replace(')', '')
hospital['lat'] = pd.to_numeric(hospital['lat'])
hospital['long'] = pd.to_numeric(hospital['long'])

<ipython-input-17-1c7e13b19131>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  hospital['lat'] = hospital['location'].str.split(pat = ", ", expand = True)[0].str.replace('(', '')
<ipython-input-17-1c7e13b19131>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  hospital['long'] =  hospital['location'].str.split(pat = ", ", expand = True)[1].str.replace(')', '')


In [18]:
better_hosp= hospital[hospital.compared_to_NR=="Better"]

In [19]:
county = better_hosp.county_name.value_counts().rename_axis('County').reset_index(name='counts')
county

,County,counts
0,LOS ANGELES,74
1,COOK,50
...,...,...
320,GWINNETT,1
321,COLLIN,1


In [20]:
county = county[county.counts >10]
county

,County,counts
0,LOS ANGELES,74
1,COOK,50
...,...,...
16,MIDDLESEX,11
17,OAKLAND,11


In [21]:
worse_hosp= hospital[hospital.compared_to_NR=="Worse"]
worse_hosp

,providerID,hospital_name,address,city,state,zip_code,county_name,measureName,measureID,compared_to_NR,denominator,score,location,rateType,condition,lat,long
5,230130,"BEAUMONT HOSPITAL, ROYAL OAK",3601 W THIRTEEN MILE RD,ROYAL OAK,MI,48073,OAKLAND,Pneumonia (PN) 30-Day Readmission Rate,READM_PN,Worse,1896,20.6,"(42.517687, -83.192142)",READM,PN,42.517687,-83.192142
8,230130,"BEAUMONT HOSPITAL, ROYAL OAK",3601 W THIRTEEN MILE RD,ROYAL OAK,MI,48073,OAKLAND,Heart failure (HF) 30-Day Readmission Rate,READM_HF,Worse,2371,23.9,"(42.517687, -83.192142)",READM,HF,42.517687,-83.192142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63823,530006,SHERIDAN MEMORIAL HOSPITAL,1401 W 5TH ST,SHERIDAN,WY,82801,SHERIDAN,Heart failure (HF) 30-Day Mortality Rate,MORT_HF,Worse,79,16.3,"(44.807425, -106.97557)",MORT,HF,44.807425,-106.975570
64010,521334,THEDACARE MEDICAL CENTER - WAUPACA INC,800 RIVERSIDE DRIVE,WAUPACA,WI,54981,WAUPACA,Pneumonia (PN) 30-Day Mortality Rate,MORT_PN,Worse,141,22.8,"(44.345495, -89.076214)",MORT,PN,44.345495,-89.076214


In [22]:
county_w = worse_hosp.county_name.value_counts().rename_axis('County').reset_index(name='counts')
county_w

,County,counts
0,COOK,35
1,WAYNE,29
...,...,...
437,MESA,1
438,WAUPACA,1


In [23]:
county_w = county_w[county_w.counts> 10]
county_w

,County,counts
0,COOK,35
1,WAYNE,29
...,...,...
24,SAINT LOUIS,11
25,OAKLAND,11


### Adjustment to Other Data Sets

For this project, we are looking into factors that can influence a hospital's performance. <br>
So, we've introduced data sets pertaining to county population ('pop'), poverty ('pov'), and median income ('income'). <br>
We'll now clean them to be able to use them.

In [24]:
pop = pop.rename(columns = {'FIPS Code': 'FIPS_code', 'State': 'state', 'Area name': 'Area_name', 'Population 2020': 'Population_2020'})
pop.head()

,FIPS_code,state,Area_name,Population_2020
0,0,US,United States,"331,449,281"
1,1000,AL,Alabama,"5,024,279"
2,1001,AL,Autauga County,"58,805"
3,1003,AL,Baldwin County,"231,767"
4,1005,AL,Barbour County,"25,223"


In [25]:
pov = pov.rename(columns = {'Stabr': 'state'})
pov.head()

,FIPS_code,state,Area_name,POVALL_2020,PCTPOVALL_2020
0,0,US,United States,"38,371,394",11.9
1,1000,AL,Alabama,"714,568",14.9
2,1001,AL,Autauga County,"6,242",11.2
3,1003,AL,Baldwin County,"20,189",8.9
4,1005,AL,Barbour County,"5,548",25.5


In [26]:
income = income.rename(columns = {'State': 'state'})
income['Area_name'] = income['Area_name'].str.split(pat = ",", expand = True)[0]
income.head()

,FIPS_code,state,Area_name,Median_Household_Income_2020
0,0.0,US,United States,"67,340"
1,1000.0,AL,Alabama,"53,958"
2,1001.0,AL,Autauga County,"67,565"
3,1003.0,AL,Baldwin County,"71,135"
4,1005.0,AL,Barbour County,"38,866"


In [27]:
popDen = popDen.rename(columns = {'State': 'state', 'county_name': 'Area_name'})
popDen = popDen.drop(columns = {'Population'})
popDen.head()

,FIPS_code,Area_name,state,PopDensity
0,1001,Autauga County,AL,35.853419
1,1003,Baldwin County,AL,50.541504
2,1005,Barbour County,AL,11.247981
3,1007,Bibb County,AL,13.973114
4,1009,Blount County,AL,34.515816


We'll now merge them into a single dataframe called factors and change it for future use.

In [28]:
factors = pd.merge(pop, income, on = ["FIPS_code", "state", "Area_name"])
factors = pd.merge(factors, pov, on = ["FIPS_code","state", "Area_name"])
factors = pd.merge(factors, popDen, on = ["FIPS_code","state", "Area_name"])
factors['Population_2020'] = factors.Population_2020.str.replace(',', '').astype(int)
factors['Median_Household_Income_2020'] = factors.Median_Household_Income_2020.str.replace(',', '').astype(int)
factors['POVALL_2020'] = factors.POVALL_2020.str.replace(',', '').astype(int)
factors = factors.loc[factors['FIPS_code']%1000 != 0]
factors.Area_name = factors['Area_name'].str.split(pat = " County", expand = True)[0]
factors.Area_name = factors['Area_name'].str.upper()
factors = factors.rename(columns = {'Area_name': 'county_name'})
factors = factors.drop(columns = 'FIPS_code')
factors = factors.rename(columns={'Median_Household_Income_2020':'Med_Income','POVALL_2020': 'Pov_Total', 'PCTPOVALL_2020': 'Pov_Percent'})

In addition to the county factors, we will also look into an individual hospital factor - the cost for conditions at different health care providers. <br>
We will use a data set with average payment for AMI, HF, and PN to explore this.

In [29]:
pay.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$26,894","$24,890","$29,027",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2018,06/30/2021
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,835","$16,947","$18,783",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Better Mortality and Average Payment,NaN,07/01/2018,06/30/2021
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,182","$19,430","$20,653",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,NaN,07/01/2018,06/30/2021
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$21,623","$20,104","$23,309",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,04/01/2018,03/31/2021
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$25,814","$22,668","$29,475",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2018,06/30/2021


In order for future use, we will transform the 'pay' data frame.

In [30]:
pay = pay.drop(columns = {'Facility Name', 'Address', 'City', 'State', 'County Name', 'ZIP Code', 'Phone Number', 'Payment Footnote', 'Lower Estimate', 'Higher Estimate',
                    'Value of Care Footnote', 'Start Date', 'End Date', 'Denominator', 'Value of Care Display ID', 'Value of Care Display Name',
                    'Value of Care Category'}, axis = 1)
pay = pay.rename(columns = {'Facility ID': 'providerID', 'Payment Measure ID': 'condition',
                            'Payment Measure Name':'payName', 'Payment Category': 'payCompared', 'Payment': 'payment'})
pay = pay[(pay['payCompared'] != 'Number of Cases Too Small') & (pay['payCompared'] != 'Not Available')]
pay.payCompared[pay.payCompared == 'No Different Than the National Average Payment'] = 0
pay.payCompared[pay.payCompared == 'Less Than the National Average Payment'] = -1
pay.payCompared[pay.payCompared == 'Greater Than the National Average Payment'] = 1
pay = pay[(pay['condition'] != 'PAYM_90_HIP_KNEE')]
pay.condition = pay.condition.str.split(pat='_', expand = True)[2]
pay.payment = pay.payment.replace('[\$,]', '', regex=True).astype(float)

Using these data frames, we will begin to merge them with 'hospital' to form new data frames.<br>


*   **payHosp** is a data frame with average payment costs for condition at each
*   **factorHosp** is a data frame with county factors for each hospital

hospital. <br>


*Note: these merged data frames have much less, but sufficient amount of rows.*

In [31]:
payHosp = pd.merge(hospital, pay, how = 'inner', on = ['providerID','condition'])
factorHosp = pd.merge(hospital, factors, how = 'inner', on = ['county_name', 'state'])

In [32]:
payHosp.head(2)

,providerID,hospital_name,address,city,state,zip_code,county_name,measureName,measureID,compared_to_NR,denominator,score,location,rateType,condition,lat,long,payName,payCompared,payment
0,230133,OTSEGO MEMORIAL HOSPITAL,825 N CENTER AVE,GAYLORD,MI,49735,OTSEGO,Heart failure (HF) 30-Day Mortality Rate,MORT_HF,No Different,102,13.4,"(45.035374, -84.673886)",MORT,HF,45.035374,-84.673886,Payment for heart failure patients,0,16365.0
1,230133,OTSEGO MEMORIAL HOSPITAL,825 N CENTER AVE,GAYLORD,MI,49735,OTSEGO,Heart failure (HF) 30-Day Readmission Rate,READM_HF,No Different,120,20.4,"(45.035374, -84.673886)",READM,HF,45.035374,-84.673886,Payment for heart failure patients,0,16365.0


##Next, Analyzing the Data

### Comparing Between Readmission and Mortality

Let's look into how readmission and mortality relate to one another!

In [33]:
import plotly
import plotly.offline as pyo
import plotly.express as px
from plotly.offline import init_notebook_mode
#pyo.init_notebook_mode()

In [34]:
byMeasureID = hospital.groupby(by = ["rateType","condition", "measureID"]).score.agg('mean').reset_index()
byMeasureID2 = hospital.groupby(by = ["condition", "measureID"]).score.agg('std').reset_index()

> byMeasureID is a groupBy object holding means of scores.<br>
byMeasureID2 is a groupBy object holding standard deviations of scores.

In [35]:
byMeasureID

,rateType,condition,measureID,score
0,MORT,AMI,MORT_AMI,14.063933
1,MORT,CABG,MORT_CABG,3.331184
...,...,...,...,...
12,READM,PN,READM_PN,17.113525
13,READM,STK,READM_STK,12.566986


In [36]:
fig = px.bar(byMeasureID, x = 'measureID', y = 'score', color = 'condition',
       color_discrete_sequence = px.colors.qualitative.Dark24,
       title = 'Mean Rates of Readmission and Mortality',
       labels = {'score':'Mean Rate (%)'})
fig.show()

**BAR PLOT#1** - Mean Rates of Readmission and Mortality <br>

This first bar graph shows that readmission and mortality is generally separate.
They are more linked with the condition itself.<br>
This makes sense considering each condition has a different likelihood for death and tendency for readmission.

In [37]:
fig = px.bar(byMeasureID2, x = 'measureID', y = 'score', color = 'condition',
       color_discrete_sequence = px.colors.qualitative.Dark24,
       title = 'Standard Deviation of Rates of Readmission and Mortality',
       labels = {'score': 'Standard Deviation'})
fig.show()

**BAR PLOT#2** - Standard Deviation of Readmission and Mortality Rates <br>

This second bar graph shows that for each condition, hospitals are likely not going to stray away from their respective average. <br>
This once again reinforces the idea that readmission and mortality rates are more connected with the condition itself rather than the hospital.

**Key Point:**
*   Readmission doesn't always correlate with mortality
*   Readmission and mortality rates are highly linked with the condition



### Comparing Between Condition

Next, let's see how different conditions compare to one another!

In [38]:
fig = px.bar(byMeasureID[byMeasureID.rateType=="READM"], x = 'measureID', y = 'score', color = 'condition',
       color_discrete_sequence = px.colors.qualitative.Dark24,
       title = 'Mean Rates of Readmission by Condition',
       labels = {'score': 'Mean Rate (%)'})
fig = fig.update_layout(xaxis={'categoryorder':'total ascending'})

In [39]:
fig1 = px.bar(byMeasureID[byMeasureID.rateType=="MORT"], x = 'measureID', y = 'score', color = 'condition',
       color_discrete_map = {'AMI': '#2E91E5', 'CABG': '#E15F99',
                             'COPD': '#1CA71C', 'HF': '#FB0D0D',
                             'PN': '#B68100', 'STK': '#750D86'},
       title = 'Mean Rates of Mortality by Condition',
       labels = {'score': 'Mean Rate (%)'})
fig1.update_layout(xaxis={'categoryorder':'total ascending'})
fig.show()
fig1.show()

**BAR PLOT#3,4** - Mean Rates of Readmission by Condition, Mean Rates of Mortality by Condition <br>
1. **Pneumonia** had the highest rate of mortality and the third most highest rate of readmission.
2. Besides pneumonia, **AMI, STK, HF, and COPD** also appeared to have high rates of readmission and/or mortality.


Seeing as hospitals would like to avoid both patient readmission and mortality, <br> they should consider especially attending to pneumonia, and the other conditions listed as well.

###Comparing Rates Based on External Factors
Are there any larger-scale factors that could be impacting a hospital's performance? <br> What kind of hospitals should be given special attention to on a nation-wide scale?




####County Distribution of Pneumonia Mortality Rates

In [40]:
tempWide = hospital[(hospital.condition == 'PN') & (hospital.rateType == 'MORT')]
byCounty = tempWide.groupby(by = ["county_name"]).score.agg('mean').reset_index()
byCounty = byCounty.sort_values(by = 'score')

In [41]:
px.bar(byCounty, x = 'county_name', y = 'score',
       title = 'County Distribution of Pneumonia Mortality Rates',
       labels = {'score':'Average Pneumonia Mortality Rate (%)', 'county_name': 'County Name'})

**BAR PLOT#5** - County Distribution of Pneumonia Mortality Rates <br>
When making distributions based on county like these, most plots are like above:

*   the majority of the middle is flat
*   the plot curls out towards the extremes

However, as can be seen, counties can differ in rates up to a difference of 10% and higher.





####Map Visualization

To visualize these hospitals on a national-scale, we'll use the lat and long values to plot them on a map

In [42]:
hospWide = factorHosp[(factorHosp.condition == 'HOSP') & (factorHosp.rateType == 'READM')]

In [43]:
fig = px.scatter_mapbox(hospWide, lat = 'lat', lon = 'long', zoom = 2.9,
                        color = 'compared_to_NR',
                        title = 'US Map of Hospital Locations and Hospital-Wide Readmission Compared to National Rate',
                        labels = {'0': 'Same as', '-1': 'Worse than', '1': 'Better than',
                                  'compared_to_NR': '  Compared to National Rate  '},
                        hover_name = 'hospital_name', hover_data = ['score'])
# can add labels to make it cleaner
fig.update_layout(mapbox_style = "carto-positron")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.96,
    xanchor="left",
    x=0.01
))
fig.show()

####Factor: Median Household Income
Does median household income affect hospital-wide readmission rate?

In [44]:
fig4 = px.scatter(hospWide, x = 'Med_Income', y= 'score',
                  title = 'Median Household Income and Hospital-Wide Readmission Rate Distribution',
                  labels = {'score': 'Hospital-Wide Readmission Rate (%)', 'Med_Income': 'Median Household Income ($)'},
                  trendline="ols", )
fig4.show()

**Scatter Plot #1** - Median Household Income and Hospital-Wide Readmission Rate Distribution


Median household income does influence hospital-wide readmission, specifically  as income increases, hospital-wide readmission decreases. <br> This is most clearly visible in the range of income up to $50,000.




#### Factor: Population Density
Does median household income affect hospital-wide readmission rate?

In [45]:
px.histogram(county, x = 'County', y='counts', color = 'County', title= "Hospital locations and Readmission Rate: Better than National Average")

In [46]:
px.histogram(county_w, x = 'County', y='counts', color = 'County' , title= "Hospital locations and Readmission Rate: Worse than National Average" )

In [47]:
fig4 = px.scatter(hospWide, x = 'PopDensity', y= 'score',
                  title = 'County Population Density and Hospital-Wide Readmission Rate Distribution',
                  labels = {'score': 'Hospital-Wide Readmission Rate (%)', 'PopDensity': 'County Population Density (# of people per sq km)'},
                  trendline="ols")
fig4.update_xaxes(range=[0, 3000])
fig4.show()


In [48]:
extreme = hospWide[hospWide.compared_to_NR != 'No Different']
fig5 = px.histogram(extreme, x = 'PopDensity', nbins = 1000,
                    title = "Extremes of Hospital-Wide Readmission Population Density Distribution",
                    labels = {'PopDensity': 'County Population Density (# of people per sq km)'})
fig5.update_xaxes(range=[0, 5000])
fig5.update_layout(yaxis_title="# of Hospitals Worse or Better than National Rate")

**Scatter Plot #2** - County Population Density and Hospital-Wide Readmission Rate Distribution


County population density does influence hospital-wide readmission, specifically as county population density increases, hospital-wide readmission generally increases too. <br>

####Factor: Cost of Treating Condition

In [49]:
payHosp.head()

,providerID,hospital_name,address,city,state,zip_code,county_name,measureName,measureID,compared_to_NR,denominator,score,location,rateType,condition,lat,long,payName,payCompared,payment
0,230133,OTSEGO MEMORIAL HOSPITAL,825 N CENTER AVE,GAYLORD,MI,49735,OTSEGO,Heart failure (HF) 30-Day Mortality Rate,MORT_HF,No Different,102,13.4,"(45.035374, -84.673886)",MORT,HF,45.035374,-84.673886,Payment for heart failure patients,0,16365.0
1,230133,OTSEGO MEMORIAL HOSPITAL,825 N CENTER AVE,GAYLORD,MI,49735,OTSEGO,Heart failure (HF) 30-Day Readmission Rate,READM_HF,No Different,120,20.4,"(45.035374, -84.673886)",READM,HF,45.035374,-84.673886,Payment for heart failure patients,0,16365.0
2,230130,"BEAUMONT HOSPITAL, ROYAL OAK",3601 W THIRTEEN MILE RD,ROYAL OAK,MI,48073,OAKLAND,Pneumonia (PN) 30-Day Readmission Rate,READM_PN,Worse,1896,20.6,"(42.517687, -83.192142)",READM,PN,42.517687,-83.192142,Payment for pneumonia patients,1,20687.0
3,230130,"BEAUMONT HOSPITAL, ROYAL OAK",3601 W THIRTEEN MILE RD,ROYAL OAK,MI,48073,OAKLAND,Pneumonia (PN) 30-Day Mortality Rate,MORT_PN,Better,1748,12.8,"(42.517687, -83.192142)",MORT,PN,42.517687,-83.192142,Payment for pneumonia patients,1,20687.0
4,230130,"BEAUMONT HOSPITAL, ROYAL OAK",3601 W THIRTEEN MILE RD,ROYAL OAK,MI,48073,OAKLAND,Heart failure (HF) 30-Day Readmission Rate,READM_HF,Worse,2371,23.9,"(42.517687, -83.192142)",READM,HF,42.517687,-83.192142,Payment for heart failure patients,0,18904.0


In [50]:
tempWide = payHosp[payHosp.measureID == 'READM_HF']

In [51]:
fig4 = px.scatter(tempWide, x = 'payment', y= 'score',
                  title = 'Distribution of Median Household Income and Hospital-Wide Readmission Rate',
                  labels = {'score': 'Readmission Rate (%)', 'Med_Income': 'Median Household Income ($)'})
fig4.show()

##Finally, Predictive Modeling with Decision Trees

In [52]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [53]:
modelHosp = hospWide.drop(columns = {'providerID', 'hospital_name', 'address', 'city', 'zip_code', 'state', 'county_name',
                                       'measureName', 'score', 'location', 'lat', 'long', 'measureID', 'denominator', 'Population_2020',
                                     'rateType', 'condition'})
modelHosp.loc[modelHosp["compared_to_NR"] == "No Different", "compared_to_NR"] = 0
modelHosp.loc[modelHosp["compared_to_NR"] == "Worse", "compared_to_NR"] = 1
modelHosp.loc[modelHosp["compared_to_NR"] == "Better", "compared_to_NR"] = 0
modelHosp["compared_to_NR"] = modelHosp["compared_to_NR"].astype('int')

In [54]:
modelHosp.head()

,compared_to_NR,Med_Income,Pov_Total,Pov_Percent,PopDensity
0,0,43450,3420,13.8,17.752377
12,0,59733,7772,11.6,49.806630
27,0,53616,3500,11.6,14.567191
34,0,53616,3500,11.6,14.567191
37,0,53616,3500,11.6,14.567191


In [55]:
X = modelHosp.drop(columns = 'compared_to_NR')
y = modelHosp['compared_to_NR']

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [58]:
predicted = clf.predict(X_test)
actual = np.array(y_test)

print('Look at first 10 predictions:')
print('Predicted: ',predicted[:10])
print('Actual:    ',actual[:10])

Look at first 10 predictions:
Predicted:  [0 0 0 0 0 0 0 0 0 1]
Actual:     [0 0 0 0 0 0 1 0 0 0]


In [59]:
from sklearn.metrics import accuracy_score
accuracy_tree = accuracy_score(y_true = actual, y_pred = predicted)
print(accuracy_tree)

0.9287305122494433


In [60]:
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:, 1]

In [61]:
test_results = pd.DataFrame(data = {'Actual Class':y_test, 'Predicted Class':y_pred, 'Predicted Probabilty':y_pred_proba})
test_results.sample(5)

,Actual Class,Predicted Class,Predicted Probabilty
1929,0,0,0.000000
33271,0,0,0.000000
19196,0,1,1.000000
33108,0,0,0.000000
11645,0,0,0.428571


In [62]:
y_pred_proba = pd.Series(y_pred_proba)

In [63]:
def custom_confusion_matrix(y_test_, y_pred_proba_, alpha=0.5, output='dataframe'):
    """
    Usage:
        cm = custom_confusion_matrix(y_test, y_pred_proba, output = 'dataframe')
        tn, fp, fn, tp = custom_confusion_matrix(y_test, y_pred_proba, output = 'rates')

    Params:
        alpha: Threshold probability for classification (default = 0.5)
        output: One of 'dataframe', 'rates', or 'array'
    """
    y_pred_ = (y_pred_proba_ >  alpha).map({True:1,False:0})
    cf_mat_ = confusion_matrix(y_test_, y_pred_)
    if output == 'dataframe':
        return pd.DataFrame(cf_mat_, columns=['Predicted 0', 'Predicted 1'], index=['Actual 0', 'Actual 1'])
    elif output == 'rates':
        return cf_mat_.ravel()
    else:
        return cf_mat_

In [64]:
cm = custom_confusion_matrix(y_test, y_pred_proba)
cm

,Predicted 0,Predicted 1
Actual 0,1235,33
Actual 1,63,16


In [65]:
%%shell
jupyter nbconvert --to html ///content/hospitalReadmissionMortality.ipynb

[NbConvertApp] WARNING | pattern '///content/hospitalReadmissionMortality.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.an

CalledProcessError: ignored